# Encoding de Medicamentos para Homologación

Este notebook implementa un sistema de encoding especializado para medicamentos destinado a facilitar la homologación y clustering.

## Importación de Librerías

Librerías necesarias para el procesamiento de datos y encoding de variables categóricas y numéricas.

In [57]:
import os
import polars as pl
import pandas as pd
import numpy as np
from collections import Counter


## Carga del Dataset

Carga del dataset de medicamentos preprocesado desde archivo parquet para iniciar el proceso de encoding.

In [58]:
# Cargar dataset original
df_medicamentos = pl.read_parquet('./data/medicamentos_preprocesados.parquet')

print(f"\n📊 Dataset cargado:")
print(f"   - Shape: {df_medicamentos.shape}")
print(f"   - Columnas: {df_medicamentos.columns}")
df_medicamentos


📊 Dataset cargado:
   - Shape: (248635, 15)
   - Columnas: ['CUM', 'PRODUCTO', 'EXPEDIENTE CUM', 'ATC', 'DESCRIPCIÓN_ATC', 'VÍA ADMINISTRACIÓN', 'PRINCIPIO ACTIVO', 'FORMA FARMACÉUTICA', 'CANTIDAD CUM', 'CANTIDAD', 'UNIDAD MEDIDA', 'VALIDO', 'ESTADO REGISTRO', 'ESTADO CUM', 'MUESTRA MÉDICA']


CUM,PRODUCTO,EXPEDIENTE CUM,ATC,DESCRIPCIÓN_ATC,VÍA ADMINISTRACIÓN,PRINCIPIO ACTIVO,FORMA FARMACÉUTICA,CANTIDAD CUM,CANTIDAD,UNIDAD MEDIDA,VALIDO,ESTADO REGISTRO,ESTADO CUM,MUESTRA MÉDICA
str,str,i64,str,str,str,str,str,f64,f64,str,i8,str,str,str
"""10042-1""","""FLUNARICINA 10 MG""",10042,"""N07CA03""","""FLUNARIZINA""","""ORAL""","""FLUNARICINA""","""TABLETA""",20.0,10.0,"""mg""",0,"""Vencido""","""Inactivo""","""No"""
"""10045-1""","""IBUPROFENO 500 MG TABLETAS""",10045,"""M01AE01""","""IBUPROFENO""","""ORAL""","""IBUPROFENO""","""TABLETA""",10.0,400.0,"""mg""",0,"""Vencido""","""Inactivo""","""No"""
"""10045-2""","""IBUPROFENO 500 MG TABLETAS""",10045,"""M01AE01""","""IBUPROFENO""","""ORAL""","""IBUPROFENO""","""TABLETA""",30.0,400.0,"""mg""",0,"""Vencido""","""Inactivo""","""No"""
"""100454-1""","""CLOTRIMAZOL 100 MG""",100454,"""G01AF02""","""CLOTRIMAZOL""","""VAGINAL""","""CLOTRIMAZOL""","""TABLETA VAGINAL""",6.0,100.0,"""mg""",0,"""Vencido""","""Inactivo""","""No"""
"""100458-1""","""DICLOFENACO 50 MG TABLETAS""",100458,"""M01AB05""","""DICLOFENACO""","""ORAL""","""DICLOFENACO SÓDICO""","""TABLETA CUBIERTA CON PELICULA""",100.0,50.0,"""mg""",0,"""Vencido""","""Inactivo""","""No"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""9870-5""","""CIPROFLOXACINA TABLETAS 500 MG""",9870,"""J01MA02""","""CIPROFLOXACINA""","""ORAL""","""CIPROFLOXACINA CLORHIDRATO MON…","""TABLETA CUBIERTA (GRAGEA)""",50.0,500.0,"""mg""",0,"""Vencido""","""Inactivo""","""No"""
"""9870-6""","""CIPROFLOXACINA TABLETAS 500 MG""",9870,"""J01MA02""","""CIPROFLOXACINA""","""ORAL""","""CIPROFLOXACINA CLORHIDRATO MON…","""TABLETA CUBIERTA (GRAGEA)""",100.0,500.0,"""mg""",0,"""Vencido""","""Inactivo""","""No"""
"""9870-7""","""CIPROFLOXACINA TABLETAS 500 MG""",9870,"""J01MA02""","""CIPROFLOXACINA""","""ORAL""","""CIPROFLOXACINA CLORHIDRATO MON…","""TABLETA CUBIERTA (GRAGEA)""",200.0,500.0,"""mg""",0,"""Vencido""","""Inactivo""","""No"""


## Funciones de Análisis y Preprocesamiento

Definición de funciones auxiliares para análisis inicial del dataset y visualización de muestras de datos.

In [59]:
#funciones de análisis y preprocesamiento
def analizar_dataset_inicial(df):
    """
    Realiza análisis inicial del dataset para el sistema de homologación.
    
    Args:
        df (pl.DataFrame): Dataset de medicamentos cargado con Polars
        
    Returns:
        dict: Diccionario con estadísticas del análisis
        
    Examples:
        >>> stats = analizar_dataset_inicial(df_medicamentos)
        >>> print(f"Medicamentos válidos: {stats['validos_count']}")
    """
    print("🔍 ANÁLISIS INICIAL DEL DATASET")
    print("=" * 50)
    
    # Información básica
    print(f"📊 Shape del dataset: {df.shape}")
    print(f"💾 Memoria estimada: {df.estimated_size('mb'):.2f} MB")
    
    # Análisis de validez
    validos = df.filter(pl.col('VALIDO') == 1)
    invalidos = df.filter(pl.col('VALIDO') == 0)
    
    total_count = df.height
    validos_count = validos.height
    invalidos_count = invalidos.height
    
    print(f"\n📋 DISTRIBUCIÓN DE VALIDEZ:")
    print(f"✅ Medicamentos VÁLIDOS: {validos_count:,} ({validos_count/total_count*100:.1f}%)")
    print(f"❌ Medicamentos INVÁLIDOS: {invalidos_count:,} ({invalidos_count/total_count*100:.1f}%)")
    
    # Verificar columnas disponibles
    print(f"\n🔍 VERIFICACIÓN DE COLUMNAS:")
    columnas_disponibles = df.columns
    columnas_faltantes = []
    
    for col in COLUMNAS_ENTRENAMIENTO + COLUMNAS_INFORMATIVAS + [COLUMNA_FILTRO]:
        if col in columnas_disponibles:
            print(f"✅ {col}")
        else:
            print(f"❌ {col} - NO ENCONTRADA")
            columnas_faltantes.append(col)
    
    # Análisis de cardinalidad para columnas críticas
    print(f"\n📈 CARDINALIDAD DE VARIABLES CRÍTICAS:")
    for col in ['ATC', 'VIA_ADMINISTRACION', 'PRINCIPIO_ACTIVO']:
        if col in columnas_disponibles:
            total_unicos = df.select(pl.col(col).n_unique()).item()
            validos_unicos = validos.select(pl.col(col).n_unique()).item()
            print(f"🔸 {col}: {total_unicos:,} total | {validos_unicos:,} en válidos")
    
    # Retornar estadísticas
    stats = {
        'total_registros': total_count,
        'validos_count': validos_count,
        'invalidos_count': invalidos_count,
        'columnas_faltantes': columnas_faltantes,
        'columnas_disponibles': columnas_disponibles,
        'df_validos': validos,
        'df_invalidos': invalidos
    }
    
    return stats

def mostrar_muestra_datos(df, n_samples=5):
    """
    Muestra una muestra de los datos para verificar la estructura.
    
    Args:
        df (pl.DataFrame): Dataset de medicamentos con Polars
        n_samples (int): Número de muestras a mostrar
        
    Examples:
        >>> mostrar_muestra_datos(df_medicamentos, 3)
    """
    print(f"\n📋 MUESTRA DE DATOS ({n_samples} registros):")
    print("=" * 80)
    
    # Mostrar columnas de entrenamiento
    print("🎯 COLUMNAS DE ENTRENAMIENTO:")
    cols_disponibles = [col for col in COLUMNAS_ENTRENAMIENTO if col in df.columns]
    if cols_disponibles:
        muestra_entrenamiento = df.select(cols_disponibles + ['VALIDO']).head(n_samples)
        print(muestra_entrenamiento)
    
    print(f"\n📝 COLUMNAS INFORMATIVAS (muestra):")
    cols_info_disponibles = [col for col in COLUMNAS_INFORMATIVAS if col in df.columns]
    if cols_info_disponibles:
        muestra_informativa = df.select(cols_info_disponibles).head(n_samples)
        print(muestra_informativa)


## Configuración de Columnas y Pesos

Definición de columnas por importancia (críticas, importantes, informativas) y configuración de pesos jerárquicos para el scoring final.

In [60]:
# Definir columnas según su propósito
COLUMNAS_ENTRENAMIENTO = [
    'ATC',                    # CRÍTICO - Código terapéutico
    'VÍA ADMINISTRACIÓN',     # CRÍTICO - Vía de administración  
    'PRINCIPIO ACTIVO',       # CRÍTICO - Sustancia activa
    'FORMA FARMACÉUTICA',     # IMPORTANTE - Presentación
    'CANTIDAD CUM',           # IMPORTANTE - Dosis/concentración
    'CANTIDAD',               # IMPORTANTE - Cantidad total
    'UNIDAD MEDIDA',          # IMPORTANTE - Unidad de medida
]

COLUMNAS_INFORMATIVAS = [
    'CUM',                    # Identificador único (entrada/salida)
    'PRODUCTO',               # Nombre comercial (solo para mostrar al usuario)
    'EXPEDIENTE CUM',         # Información administrativa
    'DESCRIPCIÓN_ATC',        # Solo para consulta humana
]

COLUMNA_FILTRO = 'VALIDO'    # Para filtrar medicamentos válidos

# Pesos jerárquicos para el scoring final
PESOS_JERARQUICOS = {
    'ATC': 0.40,                    # 40% - CRÍTICO
    'VIA ADMINISTRACION': 0.25,     # 25% - CRÍTICO  
    'PRINCIPIO ACTIVO': 0.20,       # 20% - CRÍTICO
    'FORMA FARMACEUTICA': 0.10,     # 10% - IMPORTANTE
    'CANTIDAD SIMILITUD': 0.05      # 5%  - IMPORTANTE (combinación de cantidad y unidad)
}

## Ejecución del Análisis Inicial

Análisis exploratorio del dataset para entender la distribución de medicamentos válidos e inválidos y verificar la estructura de los datos.

In [61]:
# Ejecutar análisis inicial
print("🚀 INICIANDO ANÁLISIS DEL DATASET PARA HOMOLOGACIÓN DE MEDICAMENTOS")
print("=" * 70)

# Analizar el dataset (usando la variable que cargaste con polars)
stats = analizar_dataset_inicial(df_medicamentos)

# Mostrar muestra de datos
mostrar_muestra_datos(df_medicamentos, 3)

print(f"\n✅ ANÁLISIS INICIAL COMPLETADO")
print(f"📊 Medicamentos válidos disponibles para entrenamiento: {stats['validos_count']:,}")

🚀 INICIANDO ANÁLISIS DEL DATASET PARA HOMOLOGACIÓN DE MEDICAMENTOS
🔍 ANÁLISIS INICIAL DEL DATASET
📊 Shape del dataset: (248635, 15)
💾 Memoria estimada: 39.16 MB



📋 DISTRIBUCIÓN DE VALIDEZ:
✅ Medicamentos VÁLIDOS: 62,006 (24.9%)
❌ Medicamentos INVÁLIDOS: 186,629 (75.1%)

🔍 VERIFICACIÓN DE COLUMNAS:
✅ ATC
✅ VÍA ADMINISTRACIÓN
✅ PRINCIPIO ACTIVO
✅ FORMA FARMACÉUTICA
✅ CANTIDAD CUM
✅ CANTIDAD
✅ UNIDAD MEDIDA
✅ CUM
✅ PRODUCTO
✅ EXPEDIENTE CUM
✅ DESCRIPCIÓN_ATC
✅ VALIDO

📈 CARDINALIDAD DE VARIABLES CRÍTICAS:
🔸 ATC: 2,419 total | 1,568 en válidos

📋 MUESTRA DE DATOS (3 registros):
🎯 COLUMNAS DE ENTRENAMIENTO:
shape: (3, 8)
┌─────────┬──────────────┬──────────────┬──────────────┬──────────────┬──────────┬────────┬────────┐
│ ATC     ┆ VÍA ADMINIST ┆ PRINCIPIO    ┆ FORMA        ┆ CANTIDAD CUM ┆ CANTIDAD ┆ UNIDAD ┆ VALIDO │
│ ---     ┆ RACIÓN       ┆ ACTIVO       ┆ FARMACÉUTICA ┆ ---          ┆ ---      ┆ MEDIDA ┆ ---    │
│ str     ┆ ---          ┆ ---          ┆ ---          ┆ f64          ┆ f64      ┆ ---    ┆ i8     │
│         ┆ str          ┆ str          ┆ str          ┆              ┆          ┆ str    ┆        │
╞═════════╪══════════════╪══════

## Clase MedicamentosEncoder

Implementación de encoder especializado que aplica diferentes estrategias de encoding según la importancia jerárquica de las variables categóricas y numéricas.

In [62]:
import polars as pl
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from typing import Dict, Tuple, List


class MedicamentosEncoder:
    """
    Encoder especializado para medicamentos con priorización jerárquica.

    Estrategia de Encoding:
    1. Variables CRÍTICAS: Encoding directo + penalización por no coincidencia
    2. Variables IMPORTANTES: Encoding con tolerancia y scoring gradual
    3. Variables NUMÉRICAS: Normalización + cálculo de similitud

    Examples:
        >>> encoder = MedicamentosEncoder()
        >>> df_encoded = encoder.fit_transform(df_medicamentos)
    """

    def __init__(self):
        """Inicializa el encoder con configuraciones por defecto."""
        self.pesos_jerarquicos = {
            'ATC': 0.40,
            'VIA_ADMINISTRACION': 0.25,
            'PRINCIPIO_ACTIVO': 0.20,
            'FORMA_FARMACEUTICA': 0.10,
            'CANTIDAD_SIMILITUD': 0.05
        }

        # Encoders para cada variable
        self.encoders = {}
        self.scalers = {}
        self.vocabularios = {}

        # Métricas de calidad
        self.encoding_stats = {}

    def analizar_variables_categoricas(self, df: pl.DataFrame) -> Dict:
        """
        Analiza variables categóricas para diseñar estrategia de encoding.

        Args:
            df (pl.DataFrame): Dataset de medicamentos

        Returns:
            Dict: Estadísticas de cada variable categórica

        Notes:
            Analiza cardinalidad, distribución y patrones para optimizar encoding
        """
        print("🔍 ANÁLISIS DE VARIABLES CATEGÓRICAS")
        print("=" * 50)

        variables_categoricas = ['ATC', 'VÍA ADMINISTRACIÓN', 'PRINCIPIO ACTIVO',
                                 'FORMA FARMACÉUTICA', 'UNIDAD MEDIDA']

        analisis = {}

        for var in variables_categoricas:
            if var in df.columns:
                # Estadísticas generales
                total_unicos = df.select(pl.col(var).n_unique()).item()
                total_registros = df.height

                # Estadísticas en válidos
                df_validos = df.filter(pl.col('VALIDO') == 1)
                validos_unicos = df_validos.select(
                    pl.col(var).n_unique()).item()
                validos_registros = df_validos.height

                # Top valores más frecuentes
                top_valores = (df_validos
                               .group_by(var)
                               .agg(pl.len().alias('count'))
                               .sort('count', descending=True)
                               .head(5))

                # Calcular estrategia de encoding recomendada
                cardinalidad_ratio = total_unicos / total_registros

                if cardinalidad_ratio < 0.01:  # Baja cardinalidad
                    estrategia = "ONE_HOT_ENCODING"
                elif cardinalidad_ratio < 0.1:  # Media cardinalidad
                    estrategia = "LABEL_ENCODING + EMBEDDING"
                else:  # Alta cardinalidad
                    estrategia = "TARGET_ENCODING + FREQUENCY_ENCODING"

                analisis[var] = {
                    'total_unicos': total_unicos,
                    'validos_unicos': validos_unicos,
                    'cardinalidad_ratio': cardinalidad_ratio,
                    'estrategia_recomendada': estrategia,
                    'top_valores': top_valores,
                    'cobertura_validos': validos_unicos / total_unicos
                }

                print(f"🔸 {var}:")
                print(
                    f"   📊 Únicos: {total_unicos:,} total | {validos_unicos:,} válidos")
                print(f"   📈 Cardinalidad: {cardinalidad_ratio:.3f}")
                print(f"   🎯 Estrategia: {estrategia}")
                print(
                    f"   📋 Top valor: {top_valores.row(0)[0]} ({top_valores.row(0)[1]:,} registros)")
                print()

        return analisis

    def crear_encoding_critico(self, df: pl.DataFrame, columna: str) -> pl.DataFrame:
        """
        Crea encoding para variables CRÍTICAS (ATC, VÍA, PRINCIPIO ACTIVO).

        Args:
            df (pl.DataFrame): Dataset de medicamentos
            columna (str): Nombre de la columna a encodear

        Returns:
            pl.DataFrame: DataFrame con nuevas columnas encoded

        Notes:
            Variables críticas requieren coincidencia exacta para homologación
        """
        print(f"🎯 ENCODING CRÍTICO: {columna}")

        # Obtener vocabulario de medicamentos válidos
        valores_validos = (df.filter(pl.col('VALIDO') == 1)
                           .select(pl.col(columna).unique())
                           .to_series()
                           .to_list())

        # Crear mapping de frecuencias en válidos
        freq_mapping = (df.filter(pl.col('VALIDO') == 1)
                        .group_by(columna)
                        .agg(pl.len().alias('freq_validos'))
                        .with_columns(
            (pl.col('freq_validos') /
             pl.col('freq_validos').sum()).alias('prob_validos')
        ))

        # Label encoding
        valores_unicos = df.select(
            pl.col(columna).unique()).to_series().to_list()
        label_map = {val: idx for idx,
                     val in enumerate(sorted(valores_unicos))}

        # Aplicar encodings
        df_encoded = (df
                      .with_columns([
                          # Label encoding
                          pl.col(columna).map_elements(
                              lambda x: label_map.get(x, -1)).alias(f'{columna}_label'),

                          # Binary encoding: está en válidos
                          pl.col(columna).is_in(valores_validos).cast(
                              pl.Int8).alias(f'{columna}_es_valido'),
                      ])
                      .join(freq_mapping, on=columna, how='left')
                      .with_columns([
                          pl.col('freq_validos').fill_null(0),
                          pl.col('prob_validos').fill_null(0)
                      ])
                      .rename({
                          'freq_validos': f'{columna}_freq_validos',
                          'prob_validos': f'{columna}_prob_validos'
                      }))

        # Guardar encoder info
        self.encoders[columna] = {
            'tipo': 'CRITICO',
            'label_map': label_map,
            'valores_validos': valores_validos,
            'freq_mapping': freq_mapping
        }

        print(
            f"   ✅ Creadas columnas: {columna}_label, {columna}_es_valido, {columna}_freq_validos, {columna}_prob_validos")

        return df_encoded

    def crear_encoding_importante(self, df: pl.DataFrame, columna: str) -> pl.DataFrame:
        """
        Crea encoding para variables IMPORTANTES (FORMA FARMACÉUTICA).

        Args:
            df (pl.DataFrame): Dataset de medicamentos
            columna (str): Nombre de la columna a encodear

        Returns:
            pl.DataFrame: DataFrame con nuevas columnas encoded

        Notes:
            Variables importantes permiten cierta variación con scoring gradual
        """
        print(f"🔸 ENCODING IMPORTANTE: {columna}")

        # Similar al crítico pero con más tolerancia
        valores_unicos = df.select(
            pl.col(columna).unique()).to_series().to_list()
        label_map = {val: idx for idx,
                     val in enumerate(sorted(valores_unicos))}

        # Frecuencias generales (no solo válidos)
        freq_mapping = (df
                        .group_by(columna)
                        .agg(pl.len().alias('freq_total'))
                        .with_columns(
                            (pl.col('freq_total') /
                             pl.col('freq_total').sum()).alias('prob_total')
                        ))

        df_encoded = (df
                      .with_columns([
                          pl.col(columna).map_elements(
                              lambda x: label_map.get(x, -1)).alias(f'{columna}_label')
                      ])
                      .join(freq_mapping, on=columna, how='left')
                      .rename({
                          'freq_total': f'{columna}_freq',
                          'prob_total': f'{columna}_prob'
                      }))

        self.encoders[columna] = {
            'tipo': 'IMPORTANTE',
            'label_map': label_map,
            'freq_mapping': freq_mapping
        }

        print(
            f"   ✅ Creadas columnas: {columna}_label, {columna}_freq, {columna}_prob")

        return df_encoded

    def crear_encoding_numerico(self, df: pl.DataFrame) -> pl.DataFrame:
        """
        Crea encoding para variables numéricas (CANTIDAD, CANTIDAD CUM).

        Args:
            df (pl.DataFrame): Dataset de medicamentos

        Returns:
            pl.DataFrame: DataFrame con variables numéricas normalizadas

        Notes:
            Normaliza y crea features de similitud para cantidades
        """
        print(f"🔢 ENCODING NUMÉRICO: CANTIDAD, CANTIDAD CUM")

        # Log transform para manejar outliers
        df_encoded = df.with_columns([
            # Log transform (agregamos 1 para evitar log(0))
            (pl.col('CANTIDAD') + 1).log().alias('CANTIDAD_log'),
            (pl.col('CANTIDAD CUM') + 1).log().alias('CANTIDAD_CUM_log'),

            # Ratios útiles
            (pl.col('CANTIDAD') / pl.col('CANTIDAD CUM')).alias('RATIO_CANTIDAD'),

            # Binning por rangos
            pl.when(pl.col('CANTIDAD') <= 10).then(0)
              .when(pl.col('CANTIDAD') <= 100).then(1)
              .when(pl.col('CANTIDAD') <= 500).then(2)
              .otherwise(3).alias('CANTIDAD_bin')
        ])

        print(f"   ✅ Creadas columnas: CANTIDAD_log, CANTIDAD_CUM_log, RATIO_CANTIDAD, CANTIDAD_bin")

        return df_encoded

    def fit_transform(self, df: pl.DataFrame) -> pl.DataFrame:
        """
        Aplica todo el pipeline de encoding al dataset.

        Args:
            df (pl.DataFrame): Dataset original de medicamentos

        Returns:
            pl.DataFrame: Dataset completamente encoded para ML

        Raises:
            ValueError: Si faltan columnas requeridas

        Examples:
            >>> encoder = MedicamentosEncoder()
            >>> df_encoded = encoder.fit_transform(df_medicamentos)
        """
        print("🚀 INICIANDO PIPELINE DE ENCODING COMPLETO")
        print("=" * 60)

        # Análisis inicial
        analisis = self.analizar_variables_categoricas(df)

        # Aplicar encodings secuencialmente
        df_encoded = df.clone()

        # Variables críticas
        variables_criticas = ['ATC', 'VÍA ADMINISTRACIÓN', 'PRINCIPIO ACTIVO']
        for var in variables_criticas:
            if var in df.columns:
                df_encoded = self.crear_encoding_critico(df_encoded, var)

        # Variables importantes
        variables_importantes = ['FORMA FARMACÉUTICA', 'UNIDAD MEDIDA']
        for var in variables_importantes:
            if var in df.columns:
                df_encoded = self.crear_encoding_importante(df_encoded, var)

        # Variables numéricas
        df_encoded = self.crear_encoding_numerico(df_encoded)

        print(f"\n✅ ENCODING COMPLETADO")
        print(f"📊 Columnas originales: {df.width}")
        print(f"📊 Columnas después del encoding: {df_encoded.width}")
        print(f"📊 Nuevas features creadas: {df_encoded.width - df.width}")

        return df_encoded

## Aplicación del Encoding

Instanciación del encoder y aplicación del proceso completo de encoding al dataset de medicamentos.

In [63]:
# Ejecutar encoding
print("🎯 INICIANDO PROCESO DE ENCODING ESTRATÉGICO")
print("=" * 70)

# Crear encoder
encoder = MedicamentosEncoder()

# Aplicar encoding al dataset
df_encoded = encoder.fit_transform(df_medicamentos)

# Mostrar resultado
print(f"\n📋 MUESTRA DEL DATASET ENCODED:")
columnas_mostrar = ['CUM', 'ATC', 'ATC_label', 'ATC_es_valido', 'VÍA ADMINISTRACIÓN', 'PRINCIPIO ACTIVO', 'VALIDO']
print(df_encoded.select(columnas_mostrar).head(5))


🎯 INICIANDO PROCESO DE ENCODING ESTRATÉGICO
🚀 INICIANDO PIPELINE DE ENCODING COMPLETO
🔍 ANÁLISIS DE VARIABLES CATEGÓRICAS
🔸 ATC:
   📊 Únicos: 2,419 total | 1,568 válidos
   📈 Cardinalidad: 0.010
   🎯 Estrategia: ONE_HOT_ENCODING
   📋 Top valor: N02BE51 (2,540 registros)

🔸 VÍA ADMINISTRACIÓN:
   📊 Únicos: 51 total | 47 válidos
   📈 Cardinalidad: 0.000
   🎯 Estrategia: ONE_HOT_ENCODING
   📋 Top valor: ORAL (42,165 registros)

🔸 PRINCIPIO ACTIVO:
   📊 Únicos: 20,321 total | 6,075 válidos
   📈 Cardinalidad: 0.082
   🎯 Estrategia: LABEL_ENCODING + EMBEDDING
   📋 Top valor: ACETAMINOFEN (1,134 registros)

🔸 FORMA FARMACÉUTICA:
   📊 Únicos: 112 total | 104 válidos
   📈 Cardinalidad: 0.000
   🎯 Estrategia: ONE_HOT_ENCODING
   📋 Top valor: TABLETA RECUBIERTA (11,912 registros)

🔸 UNIDAD MEDIDA:
   📊 Únicos: 180 total | 85 válidos
   📈 Cardinalidad: 0.001
   🎯 Estrategia: ONE_HOT_ENCODING
   📋 Top valor: mg (46,554 registros)

🎯 ENCODING CRÍTICO: ATC
   ✅ Creadas columnas: ATC_label, ATC_es_val

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



🎯 ENCODING CRÍTICO: VÍA ADMINISTRACIÓN
   ✅ Creadas columnas: VÍA ADMINISTRACIÓN_label, VÍA ADMINISTRACIÓN_es_valido, VÍA ADMINISTRACIÓN_freq_validos, VÍA ADMINISTRACIÓN_prob_validos
🎯 ENCODING CRÍTICO: PRINCIPIO ACTIVO
   ✅ Creadas columnas: PRINCIPIO ACTIVO_label, PRINCIPIO ACTIVO_es_valido, PRINCIPIO ACTIVO_freq_validos, PRINCIPIO ACTIVO_prob_validos
🔸 ENCODING IMPORTANTE: FORMA FARMACÉUTICA
   ✅ Creadas columnas: FORMA FARMACÉUTICA_label, FORMA FARMACÉUTICA_freq, FORMA FARMACÉUTICA_prob
🔸 ENCODING IMPORTANTE: UNIDAD MEDIDA
   ✅ Creadas columnas: UNIDAD MEDIDA_label, UNIDAD MEDIDA_freq, UNIDAD MEDIDA_prob
🔢 ENCODING NUMÉRICO: CANTIDAD, CANTIDAD CUM
   ✅ Creadas columnas: CANTIDAD_log, CANTIDAD_CUM_log, RATIO_CANTIDAD, CANTIDAD_bin

✅ ENCODING COMPLETADO
📊 Columnas originales: 15
📊 Columnas después del encoding: 37
📊 Nuevas features creadas: 22

📋 MUESTRA DEL DATASET ENCODED:
shape: (5, 7)
┌──────────┬─────────┬───────────┬───────────────┬────────────────┬────────────────────┬──────

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
C:\Users\ORPHEUX\AppData\Local\Temp\ipykernel_31128\168943391.py:212: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  .with_columns([
C:\Users\ORPHEUX\AppData\Local\Temp\ipykernel_31128\168943391.py:212: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  .with_columns([


## Visualización del Dataset Encoded

Exploración del dataset después del encoding para verificar las nuevas columnas y features generadas.

In [64]:
df_encoded

CUM,PRODUCTO,EXPEDIENTE CUM,ATC,DESCRIPCIÓN_ATC,VÍA ADMINISTRACIÓN,PRINCIPIO ACTIVO,FORMA FARMACÉUTICA,CANTIDAD CUM,CANTIDAD,UNIDAD MEDIDA,VALIDO,ESTADO REGISTRO,ESTADO CUM,MUESTRA MÉDICA,ATC_label,ATC_es_valido,ATC_freq_validos,ATC_prob_validos,VÍA ADMINISTRACIÓN_label,VÍA ADMINISTRACIÓN_es_valido,VÍA ADMINISTRACIÓN_freq_validos,VÍA ADMINISTRACIÓN_prob_validos,PRINCIPIO ACTIVO_label,PRINCIPIO ACTIVO_es_valido,PRINCIPIO ACTIVO_freq_validos,PRINCIPIO ACTIVO_prob_validos,FORMA FARMACÉUTICA_label,FORMA FARMACÉUTICA_freq,FORMA FARMACÉUTICA_prob,UNIDAD MEDIDA_label,UNIDAD MEDIDA_freq,UNIDAD MEDIDA_prob,CANTIDAD_log,CANTIDAD_CUM_log,RATIO_CANTIDAD,CANTIDAD_bin
str,str,i64,str,str,str,str,str,f64,f64,str,i8,str,str,str,i64,i8,u32,f64,i64,i8,u32,f64,i64,i8,u32,f64,i64,u32,f64,i64,u32,f64,f64,f64,f64,i32
"""10042-1""","""FLUNARICINA 10 MG""",10042,"""N07CA03""","""FLUNARIZINA""","""ORAL""","""FLUNARICINA""","""TABLETA""",20.0,10.0,"""mg""",0,"""Vencido""","""Inactivo""","""No""",1975,1,31,0.0005,34,1,42165,0.680015,9234,0,0,0.0,87,37013,0.148865,131,181932,0.731723,2.397895,3.044522,0.5,0
"""10045-1""","""IBUPROFENO 500 MG TABLETAS""",10045,"""M01AE01""","""IBUPROFENO""","""ORAL""","""IBUPROFENO""","""TABLETA""",10.0,400.0,"""mg""",0,"""Vencido""","""Inactivo""","""No""",1641,1,564,0.009096,34,1,42165,0.680015,11081,1,865,0.01395,87,37013,0.148865,131,181932,0.731723,5.993961,2.397895,40.0,2
"""10045-2""","""IBUPROFENO 500 MG TABLETAS""",10045,"""M01AE01""","""IBUPROFENO""","""ORAL""","""IBUPROFENO""","""TABLETA""",30.0,400.0,"""mg""",0,"""Vencido""","""Inactivo""","""No""",1641,1,564,0.009096,34,1,42165,0.680015,11081,1,865,0.01395,87,37013,0.148865,131,181932,0.731723,5.993961,3.433987,13.333333,2
"""100454-1""","""CLOTRIMAZOL 100 MG""",100454,"""G01AF02""","""CLOTRIMAZOL""","""VAGINAL""","""CLOTRIMAZOL""","""TABLETA VAGINAL""",6.0,100.0,"""mg""",0,"""Vencido""","""Inactivo""","""No""",898,1,245,0.003951,50,1,903,0.014563,6294,1,460,0.007419,104,425,0.001709,131,181932,0.731723,4.615121,1.94591,16.666667,1
"""100458-1""","""DICLOFENACO 50 MG TABLETAS""",100458,"""M01AB05""","""DICLOFENACO""","""ORAL""","""DICLOFENACO SÓDICO""","""TABLETA CUBIERTA CON PELICULA""",100.0,50.0,"""mg""",0,"""Vencido""","""Inactivo""","""No""",1628,1,241,0.003887,34,1,42165,0.680015,7301,1,42,0.000677,93,14395,0.057896,131,181932,0.731723,3.931826,4.615121,0.5,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""9870-5""","""CIPROFLOXACINA TABLETAS 500 MG""",9870,"""J01MA02""","""CIPROFLOXACINA""","""ORAL""","""CIPROFLOXACINA CLORHIDRATO MON…","""TABLETA CUBIERTA (GRAGEA)""",50.0,500.0,"""mg""",0,"""Vencido""","""Inactivo""","""No""",1162,1,152,0.002451,34,1,42165,0.680015,4952,0,0,0.0,92,3570,0.014358,131,181932,0.731723,6.216606,3.931826,10.0,2
"""9870-6""","""CIPROFLOXACINA TABLETAS 500 MG""",9870,"""J01MA02""","""CIPROFLOXACINA""","""ORAL""","""CIPROFLOXACINA CLORHIDRATO MON…","""TABLETA CUBIERTA (GRAGEA)""",100.0,500.0,"""mg""",0,"""Vencido""","""Inactivo""","""No""",1162,1,152,0.002451,34,1,42165,0.680015,4952,0,0,0.0,92,3570,0.014358,131,181932,0.731723,6.216606,4.615121,5.0,2
"""9870-7""","""CIPROFLOXACINA TABLETAS 500 MG""",9870,"""J01MA02""","""CIPROFLOXACINA""","""ORAL""","""CIPROFLOXACINA CLORHIDRATO MON…","""TABLETA CUBIERTA (GRAGEA)""",200.0,500.0,"""mg""",0,"""Vencido""","""Inactivo""","""No""",1162,1,152,0.002451,34,1,42165,0.680015,4952,0,0,0.0,92,3570,0.014358,131,181932,0.731723,6.216606,5.303305,2.5,2


## Preparación del Dataset de Entrenamiento

Funciones para crear el dataset final optimizado para clustering, incluyendo selección de features y creación de variables de interacción.

In [65]:
import polars as pl
import numpy as np

def crear_dataset_entrenamiento(df_encoded: pl.DataFrame) -> pl.DataFrame:
    """
    Crea el dataset final optimizado para clustering de homologación.
    
    Args:
        df_encoded (pl.DataFrame): Dataset con encoding aplicado
        
    Returns:
        pl.DataFrame: Dataset listo para entrenamiento con features seleccionadas
        
    Notes:
        - Selecciona solo features relevantes para clustering
        - Aplica normalización final
        - Calcula features de interacción
        
    Examples:
        >>> df_train = crear_dataset_entrenamiento(df_encoded)
    """
    print("🔧 CREANDO DATASET FINAL DE ENTRENAMIENTO")
    print("=" * 50)
    
    # FEATURES CRÍTICAS (peso total: 85%)
    features_criticas = [
        'ATC_label',                      # 40% - Identificador terapéutico
        'ATC_es_valido',                  # Crítico: debe estar en válidos
        'ATC_prob_validos',               # Probabilidad dentro de válidos
        
        'VÍA ADMINISTRACIÓN_label',       # 25% - Vía de administración
        'VÍA ADMINISTRACIÓN_es_valido',   # Crítico: debe estar en válidos
        'VÍA ADMINISTRACIÓN_prob_validos',
        
        'PRINCIPIO ACTIVO_label',         # 20% - Sustancia activa
        'PRINCIPIO ACTIVO_es_valido',     # Crítico: debe estar en válidos
        'PRINCIPIO ACTIVO_prob_validos'
    ]
    
    # FEATURES IMPORTANTES (peso total: 15%)
    features_importantes = [
        'FORMA FARMACÉUTICA_label',       # 10% - Forma de presentación
        'FORMA FARMACÉUTICA_prob',
        
        'CANTIDAD_log',                   # 5% - Dosificación
        'CANTIDAD_CUM_log',
        'RATIO_CANTIDAD',
        'CANTIDAD_bin',
        
        'UNIDAD MEDIDA_label',           # Para contexto de cantidad
        'UNIDAD MEDIDA_prob'
    ]
    
    # FEATURES INFORMATIVAS (mantener para referencia)
    features_informativas = [
        'CUM',
        'PRODUCTO', 
        'ATC',
        'VÍA ADMINISTRACIÓN',
        'PRINCIPIO ACTIVO',
        'FORMA FARMACÉUTICA',
        'CANTIDAD',
        'CANTIDAD CUM',
        'UNIDAD MEDIDA',
        'VALIDO'
    ]
    
    print(f"📊 Features críticas: {len(features_criticas)}")
    print(f"📊 Features importantes: {len(features_importantes)}")
    print(f"📊 Features informativas: {len(features_informativas)}")
    
    # Seleccionar features para entrenamiento
    todas_features = features_criticas + features_importantes + features_informativas
    features_disponibles = [f for f in todas_features if f in df_encoded.columns]
    
    df_training = df_encoded.select(features_disponibles)
    
    # Crear features de interacción críticas
    df_training = df_training.with_columns([
        # Combinación ATC + VÍA (muy importante para homologación)
        (pl.col('ATC_label').cast(pl.Utf8) + "_" + pl.col('VÍA ADMINISTRACIÓN_label').cast(pl.Utf8))
            .alias('ATC_VIA_combo'),
            
        # Score de validez crítica (debe ser 3 para medicamentos perfectamente válidos)
        (pl.col('ATC_es_valido') + pl.col('VÍA ADMINISTRACIÓN_es_valido') + pl.col('PRINCIPIO ACTIVO_es_valido'))
            .alias('score_validez_critica'),
            
        # Score de probabilidad crítica (promedio de probabilidades en válidos)
        ((pl.col('ATC_prob_validos') + pl.col('VÍA ADMINISTRACIÓN_prob_validos') + pl.col('PRINCIPIO ACTIVO_prob_validos')) / 3)
            .alias('score_prob_critica'),
            
        # Flag: medicamento ideal para homologación (todas las críticas válidas)
        (pl.col('ATC_es_valido') & pl.col('VÍA ADMINISTRACIÓN_es_valido') & pl.col('PRINCIPIO ACTIVO_es_valido'))
            .cast(pl.Int8).alias('es_ideal_homologacion')
    ])
    
    print(f"\n✅ Features de interacción creadas:")
    print(f"   🔸 ATC_VIA_combo: Combinación crítica ATC + VÍA")
    print(f"   🔸 score_validez_critica: Suma de flags de validez (0-3)")
    print(f"   🔸 score_prob_critica: Promedio de probabilidades críticas")
    print(f"   🔸 es_ideal_homologacion: Flag para medicamentos ideales")
    
    return df_training

def analizar_dataset_entrenamiento(df_training: pl.DataFrame) -> dict:
    """
    Analiza el dataset final de entrenamiento y muestra estadísticas clave.
    
    Args:
        df_training (pl.DataFrame): Dataset preparado para entrenamiento
        
    Returns:
        dict: Estadísticas del dataset de entrenamiento
    """
    print(f"\n📊 ANÁLISIS DEL DATASET DE ENTRENAMIENTO")
    print("=" * 50)
    
    # Información general
    print(f"📏 Dimensiones finales: {df_training.shape}")
    print(f"💾 Memoria estimada: {df_training.estimated_size('mb'):.2f} MB")
    
    # Análisis de medicamentos válidos vs inválidos
    validos = df_training.filter(pl.col('VALIDO') == 1)
    invalidos = df_training.filter(pl.col('VALIDO') == 0)
    
    print(f"\n📋 DISTRIBUCIÓN PARA ENTRENAMIENTO:")
    print(f"✅ Válidos: {validos.height:,} ({validos.height/df_training.height*100:.1f}%)")
    print(f"❌ Inválidos: {invalidos.height:,} ({invalidos.height/df_training.height*100:.1f}%)")
    
    # Análisis de medicamentos ideales para homologación
    ideales = df_training.filter(pl.col('es_ideal_homologacion') == 1)
    ideales_validos = ideales.filter(pl.col('VALIDO') == 1)
    
    print(f"\n🎯 MEDICAMENTOS IDEALES PARA HOMOLOGACIÓN:")
    print(f"📊 Total con variables críticas válidas: {ideales.height:,}")
    print(f"✅ De estos, que son VÁLIDOS: {ideales_validos.height:,}")
    print(f"📈 % de cobertura ideal: {ideales_validos.height/validos.height*100:.1f}% de válidos")
    
    # Análisis de scores críticos
    scores_validos = validos.select([
        pl.col('score_validez_critica').mean().alias('validez_promedio'),
        pl.col('score_prob_critica').mean().alias('prob_promedio'),
        pl.col('es_ideal_homologacion').sum().alias('ideales_count')
    ]).row(0)
    
    print(f"\n📈 SCORES EN MEDICAMENTOS VÁLIDOS:")
    print(f"🔸 Score validez crítica promedio: {scores_validos[0]:.2f}/3.0")
    print(f"🔸 Score probabilidad crítica promedio: {scores_validos[1]:.4f}")
    print(f"🔸 Medicamentos ideales: {scores_validos[2]:,}")
    
    # Top combinaciones ATC + VÍA en válidos
    top_combos = (validos
                 .group_by('ATC_VIA_combo')
                 .agg(pl.len().alias('count'))
                 .sort('count', descending=True)
                 .head(5))
    
    print(f"\n🏆 TOP COMBINACIONES ATC + VÍA EN VÁLIDOS:")
    for i in range(min(5, top_combos.height)):
        combo, count = top_combos.row(i)
        print(f"   {i+1}. {combo}: {count:,} medicamentos")
    
    # Preparar estadísticas para retorno
    stats = {
        'total_registros': df_training.height,
        'validos_count': validos.height,
        'invalidos_count': invalidos.height,
        'ideales_validos_count': ideales_validos.height,
        'score_validez_promedio': scores_validos[0],
        'score_prob_promedio': scores_validos[1],
        'top_combos': top_combos,
        'df_validos': validos,
        'df_invalidos': invalidos,
        'df_ideales_validos': ideales_validos
    }
    
    return stats

def mostrar_muestra_entrenamiento(df_training: pl.DataFrame, n_samples: int = 3):
    """
    Muestra una muestra del dataset final con las features más importantes.
    
    Args:
        df_training (pl.DataFrame): Dataset de entrenamiento
        n_samples (int): Número de muestras a mostrar
    """
    print(f"\n📋 MUESTRA DEL DATASET FINAL ({n_samples} registros):")
    print("=" * 80)
    
    # Columnas críticas para mostrar
    columnas_criticas = [
        'CUM', 'ATC', 'VÍA ADMINISTRACIÓN', 'PRINCIPIO ACTIVO', 
        'ATC_es_valido', 'VÍA ADMINISTRACIÓN_es_valido', 'PRINCIPIO ACTIVO_es_valido',
        'score_validez_critica', 'es_ideal_homologacion', 'VALIDO'
    ]
    
    columnas_disponibles = [col for col in columnas_criticas if col in df_training.columns]
    muestra = df_training.select(columnas_disponibles).head(n_samples)
    
    print("🎯 FEATURES CRÍTICAS:")
    print(muestra)
    
    # Muestra de features numéricas
    columnas_numericas = [
        'CUM', 'CANTIDAD_log', 'CANTIDAD_CUM_log', 'RATIO_CANTIDAD', 
        'score_prob_critica', 'VALIDO'
    ]
    
    columnas_num_disponibles = [col for col in columnas_numericas if col in df_training.columns]
    muestra_num = df_training.select(columnas_num_disponibles).head(n_samples)
    
    print(f"\n🔢 FEATURES NUMÉRICAS:")
    print(muestra_num)


## Ejecución de la Preparación Final

Creación del dataset de entrenamiento con análisis de estadísticas y visualización de muestras del resultado final.

In [66]:
# EJECUTAR PREPARACIÓN FINAL
print("🚀 INICIANDO PREPARACIÓN FINAL DEL DATASET")
print("=" * 70)

# Crear dataset de entrenamiento
df_training = crear_dataset_entrenamiento(df_encoded)

# Analizar dataset final
training_stats = analizar_dataset_entrenamiento(df_training)

# Mostrar muestra
mostrar_muestra_entrenamiento(df_training, 5)

print(f"\n✅ DATASET DE ENTRENAMIENTO PREPARADO")
print(f"🎯 Listo para clustering con {training_stats['ideales_validos_count']:,} medicamentos ideales")

🚀 INICIANDO PREPARACIÓN FINAL DEL DATASET
🔧 CREANDO DATASET FINAL DE ENTRENAMIENTO
📊 Features críticas: 9
📊 Features importantes: 8
📊 Features informativas: 10

✅ Features de interacción creadas:
   🔸 ATC_VIA_combo: Combinación crítica ATC + VÍA
   🔸 score_validez_critica: Suma de flags de validez (0-3)
   🔸 score_prob_critica: Promedio de probabilidades críticas
   🔸 es_ideal_homologacion: Flag para medicamentos ideales

📊 ANÁLISIS DEL DATASET DE ENTRENAMIENTO
📏 Dimensiones finales: (248635, 31)
💾 Memoria estimada: 58.18 MB

📋 DISTRIBUCIÓN PARA ENTRENAMIENTO:
✅ Válidos: 62,006 (24.9%)
❌ Inválidos: 186,629 (75.1%)

🎯 MEDICAMENTOS IDEALES PARA HOMOLOGACIÓN:
📊 Total con variables críticas válidas: 173,451
✅ De estos, que son VÁLIDOS: 62,006
📈 % de cobertura ideal: 100.0% de válidos

📈 SCORES EN MEDICAMENTOS VÁLIDOS:
🔸 Score validez crítica promedio: 3.00/3.0
🔸 Score probabilidad crítica promedio: 0.1618
🔸 Medicamentos ideales: 62,006

🏆 TOP COMBINACIONES ATC + VÍA EN VÁLIDOS:
   1. 1794_

## Guardado del Dataset Final

Exportación del dataset de entrenamiento procesado a formato parquet para su uso posterior en clustering y homologación.

In [67]:
df_training.write_parquet('./data/dataset_entrenamiento_homologacion.parquet')
df_training


ATC_label,ATC_es_valido,ATC_prob_validos,VÍA ADMINISTRACIÓN_label,VÍA ADMINISTRACIÓN_es_valido,VÍA ADMINISTRACIÓN_prob_validos,PRINCIPIO ACTIVO_label,PRINCIPIO ACTIVO_es_valido,PRINCIPIO ACTIVO_prob_validos,FORMA FARMACÉUTICA_label,FORMA FARMACÉUTICA_prob,CANTIDAD_log,CANTIDAD_CUM_log,RATIO_CANTIDAD,CANTIDAD_bin,UNIDAD MEDIDA_label,UNIDAD MEDIDA_prob,CUM,PRODUCTO,ATC,VÍA ADMINISTRACIÓN,PRINCIPIO ACTIVO,FORMA FARMACÉUTICA,CANTIDAD,CANTIDAD CUM,UNIDAD MEDIDA,VALIDO,ATC_VIA_combo,score_validez_critica,score_prob_critica,es_ideal_homologacion
i64,i8,f64,i64,i8,f64,i64,i8,f64,i64,f64,f64,f64,f64,i32,i64,f64,str,str,str,str,str,str,f64,f64,str,i8,str,i8,f64,i8
1975,1,0.0005,34,1,0.680015,9234,0,0.0,87,0.148865,2.397895,3.044522,0.5,0,131,0.731723,"""10042-1""","""FLUNARICINA 10 MG""","""N07CA03""","""ORAL""","""FLUNARICINA""","""TABLETA""",10.0,20.0,"""mg""",0,"""1975_34""",2,0.226838,0
1641,1,0.009096,34,1,0.680015,11081,1,0.01395,87,0.148865,5.993961,2.397895,40.0,2,131,0.731723,"""10045-1""","""IBUPROFENO 500 MG TABLETAS""","""M01AE01""","""ORAL""","""IBUPROFENO""","""TABLETA""",400.0,10.0,"""mg""",0,"""1641_34""",3,0.234354,1
1641,1,0.009096,34,1,0.680015,11081,1,0.01395,87,0.148865,5.993961,3.433987,13.333333,2,131,0.731723,"""10045-2""","""IBUPROFENO 500 MG TABLETAS""","""M01AE01""","""ORAL""","""IBUPROFENO""","""TABLETA""",400.0,30.0,"""mg""",0,"""1641_34""",3,0.234354,1
898,1,0.003951,50,1,0.014563,6294,1,0.007419,104,0.001709,4.615121,1.94591,16.666667,1,131,0.731723,"""100454-1""","""CLOTRIMAZOL 100 MG""","""G01AF02""","""VAGINAL""","""CLOTRIMAZOL""","""TABLETA VAGINAL""",100.0,6.0,"""mg""",0,"""898_50""",3,0.008644,1
1628,1,0.003887,34,1,0.680015,7301,1,0.000677,93,0.057896,3.931826,4.615121,0.5,1,131,0.731723,"""100458-1""","""DICLOFENACO 50 MG TABLETAS""","""M01AB05""","""ORAL""","""DICLOFENACO SÓDICO""","""TABLETA CUBIERTA CON PELICULA""",50.0,100.0,"""mg""",0,"""1628_34""",3,0.228193,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1162,1,0.002451,34,1,0.680015,4952,0,0.0,92,0.014358,6.216606,3.931826,10.0,2,131,0.731723,"""9870-5""","""CIPROFLOXACINA TABLETAS 500 MG""","""J01MA02""","""ORAL""","""CIPROFLOXACINA CLORHIDRATO MON…","""TABLETA CUBIERTA (GRAGEA)""",500.0,50.0,"""mg""",0,"""1162_34""",2,0.227489,0
1162,1,0.002451,34,1,0.680015,4952,0,0.0,92,0.014358,6.216606,4.615121,5.0,2,131,0.731723,"""9870-6""","""CIPROFLOXACINA TABLETAS 500 MG""","""J01MA02""","""ORAL""","""CIPROFLOXACINA CLORHIDRATO MON…","""TABLETA CUBIERTA (GRAGEA)""",500.0,100.0,"""mg""",0,"""1162_34""",2,0.227489,0
1162,1,0.002451,34,1,0.680015,4952,0,0.0,92,0.014358,6.216606,5.303305,2.5,2,131,0.731723,"""9870-7""","""CIPROFLOXACINA TABLETAS 500 MG""","""J01MA02""","""ORAL""","""CIPROFLOXACINA CLORHIDRATO MON…","""TABLETA CUBIERTA (GRAGEA)""",500.0,200.0,"""mg""",0,"""1162_34""",2,0.227489,0
